# Part 1

In [1]:
import re
import itertools

xy_re = re.compile(r'x=(\d+), y=(\d+)..(\d+)')
yx_re = re.compile(r'y=(\d+), x=(\d+)..(\d+)')

def parse_map(text):
    ''' Parse text and return a sparse matrix of clay locations, represented
    as a dict where key is (x,y) and value is the type of material there, i.e. '#'. '''
    map_ = {(500,0): '+'}
    for line in text.split('\n'):
        match = xy_re.match(line)
        if match:
            x = int(match.group(1))
            x_iter = range(x, x+1)
            ymin, ymax = int(match.group(2)), int(match.group(3))
            y_iter = range(ymin, ymax+1)
        else:
            match = yx_re.match(line)
            if match:
                y = int(match.group(1))
                y_iter = range(y, y+1)
                xmin, xmax = int(match.group(2)), int(match.group(3))
                x_iter = range(xmin, xmax+1)  
            else:
                raise Exception('Failed to parse line: {}'.format(line))
        for x,y in itertools.product(x_iter, y_iter):
            map_[x,y] = '#'
    return map_

def render_map(map_):
    ''' Render a section of the [infinite] map. '''
    min_x = min(coord[0] for coord in map_.keys()) - 1
    max_x = max(coord[0] for coord in map_.keys()) + 1
    min_y = 0 # By problem definition
    max_y = max(coord[1] for coord in map_.keys())
    # Print map header: the x coordinates
    render = ''
    coords = ['{:4}'.format(x) for x in range(min_x, max_x+1)]
    for i in range(4):
        render += '     {}\n'.format(''.join(coord[i] for coord in coords))
    # Print rows
    for y in range(min_y, max_y+1):
        render += '{:4} {}\n'.format(y, 
            ''.join(map_.get((x,y),'.') for x in range(min_x, max_x+1)))
    return render

In [335]:
def fill_water(map_, max_rounds=5_000, debug=False):
    ''' Returns a copy of the map with water filled in following the rules of the 
    problem. '''
    map_ = dict(map_)
    max_y = max(coord[1] for coord in map_.keys())
    remaining = [(500,0)]
    done = set()

    def fill_left(x, y):
        ''' Fill water starting at (x,y) and going left. Return last checked coord. '''
        while True:
            if map_.get((x, y)) == '#':
                break
            map_[x, y] = '~'
            if map_.get((x, y+1)) in (None, '|'):
                break
            x -= 1
        return x, y

    def fill_right(x, y):
        ''' Fill water starting at (x,y) and going right. Return last checked coord. '''
        while True:
            if map_.get((x, y)) == '#':
                break
            map_[x, y] = '~'
            if map_.get((x, y+1)) in (None, '|'):
                break
            x += 1
        return x, y

    def toggle_water(x, y):
        ''' Change ~ to | to the left and right of (x,y). '''
        start_x, start_y = x, y
        while True:
            if map_.get((x, y)) == '~':
                map_[x,y] = '|'
                x -= 1
            else:
                break
        x, y = start_x + 1, start_y
        while True:
            if map_.get((x, y)) == '~':
                map_[x,y] = '|'
                x += 1
            else:
                break
        
    while remaining:
        remaining.sort(key=lambda v: (v[1], v[0]))
        x, y = remaining.pop(0)
        done.add((x, y))
        down_y = y+1

        # Fill water down until it hits clay or goes off the map.
        while True:
            if down_y > max_y or map_.get((x, down_y)) == '#':
                break
            map_[x, down_y] = '|'
            down_y += 1
        
        if down_y > max_y:
            continue
            
        # Now spread out left and right. If we hit a wall on both sides,
        # go up one level and repeat. Break when we reach a level where
        # one or both sides has no wall.
        while True:
            down_y -= 1
            last_left = fill_left(x, down_y)
            last_right = fill_right(x, down_y)
#             print(last_left, last_right)
            if map_.get(last_left) == '#' and map_.get(last_right) == '#':
                # We hit walls on both sides. Repeat with the next highest level
                continue
            all_ = set(remaining) | done
            if map_.get(last_left) == '~' and last_left not in all_:
                remaining.append(last_left)
            if map_.get(last_right) == '~' and last_right not in all_:
                remaining.append(last_right)
            toggle_water(x, down_y)
            break
            
        if debug:
            print('done', done)
            print('remaining', list(sorted(remaining, key=lambda v: (v[1], v[0]))))
            print(render_map(map_))
        
        max_rounds -= 1
        if max_rounds == 0:
            print('Too many rounds! Returning early')
            break
        
    return map_

In [336]:
test_text = '''x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504'''
test_map = parse_map(test_text)
print(render_map(test_map))

                   
     44444455555555
     99999900000000
     45678901234567
   0 ......+.......
   1 ............#.
   2 .#..#.......#.
   3 .#..#..#......
   4 .#..#..#......
   5 .#.....#......
   6 .#.....#......
   7 .#######......
   8 ..............
   9 ..............
  10 ....#.....#...
  11 ....#.....#...
  12 ....#.....#...
  13 ....#######...



In [337]:
test_filled_map = fill_water(test_map)
print(render_map(test_filled_map))

                   
     44444455555555
     99999900000000
     45678901234567
   0 ......+.......
   1 ......|.....#.
   2 .#..#||||...#.
   3 .#..#~~#|.....
   4 .#..#~~#|.....
   5 .#~~~~~#|.....
   6 .#~~~~~#|.....
   7 .#######|.....
   8 ........|.....
   9 ...|||||||||..
  10 ...|#~~~~~#|..
  11 ...|#~~~~~#|..
  12 ...|#~~~~~#|..
  13 ...|#######|..



In [338]:
def count_water(map_, min_y, max_y):
    ''' Count the number of squares containing water, i.e. '~', subject to
    the rule regarding min and max y values. '''
    count = 0
    for (x,y), char in map_.items():
        if char in ('~','|') and (min_y <= y <= max_y):
            count += 1
    return count

In [339]:
count_water(test_filled_map, 1, 13)

57

In [340]:
# This scenario was originally broken.
test_text = '''x=497, y=3..7
y=7, x=497..503
x=503, y=3..7
x=493, y=10..13
x=507, y=10..13
y=13, x=493..507'''
test_map = parse_map(test_text)
test_filled_map = fill_water(test_map, max_rounds=4)
print(render_map(test_filled_map))

                        
     4444444445555555555
     9999999990000000000
     1234567890123456789
   0 .........+.........
   1 .........|.........
   2 .....|||||||||.....
   3 .....|#~~~~~#|.....
   4 .....|#~~~~~#|.....
   5 .....|#~~~~~#|.....
   6 .....|#~~~~~#|.....
   7 .....|#######|.....
   8 .....|.......|.....
   9 .|||||||||||||||||.
  10 .|#~~~~~~~~~~~~~#|.
  11 .|#~~~~~~~~~~~~~#|.
  12 .|#~~~~~~~~~~~~~#|.
  13 .|###############|.



In [341]:
# Another broken scenario.
test_text = '''x=497, y=3..5
x=503, y=3..5
y=5, x=497..503
x=493, y=8..10
x=500, y=8..10
y=10, x=493..500
x=490, y=13..15
x=506, y=13..15
y=15, x=490..506'''
test_map = parse_map(test_text)
test_filled_map = fill_water(test_map, max_rounds=7)
print(render_map(test_filled_map))

                          
     444444444444555555555
     889999999999000000000
     890123456789012345678
   0 ............+........
   1 ............|........
   2 ........|||||||||....
   3 ........|#~~~~~#|....
   4 ........|#~~~~~#|....
   5 ........|#######|....
   6 ........|.......|....
   7 ....||||||||||..|....
   8 ....|#~~~~~~#|..|....
   9 ....|#~~~~~~#|..|....
  10 ....|########|..|....
  11 ....|........|..|....
  12 .|||||||||||||||||||.
  13 .|#~~~~~~~~~~~~~~~#|.
  14 .|#~~~~~~~~~~~~~~~#|.
  15 .|#################|.



In [342]:
# Another broken scenario.
test_text = '''x=497, y=3..5
x=503, y=3..5
y=5, x=497..503
x=490, y=8..13
x=510, y=8..13
y=13, x=490..510
x=502, y=9..11
x=505, y=9..11
y=9, x=502..505
y=11, x=502..505'''
test_map = parse_map(test_text)
test_filled_map = fill_water(test_map)
print(render_map(test_filled_map))

                              
     4444444444445555555555555
     8899999999990000000000111
     8901234567890123456789012
   0 ............+............
   1 ............|............
   2 ........|||||||||........
   3 ........|#~~~~~#|........
   4 ........|#~~~~~#|........
   5 ........|#######|........
   6 ........|.......|........
   7 .|||||||||||||||||||||||.
   8 .|#~~~~~~~~~~~~~~~~~~~#|.
   9 .|#~~~~~~~~~~~####~~~~#|.
  10 .|#~~~~~~~~~~~#..#~~~~#|.
  11 .|#~~~~~~~~~~~####~~~~#|.
  12 .|#~~~~~~~~~~~~~~~~~~~#|.
  13 .|#####################|.



In [343]:
# Another broken scenario.
test_text = '''x=497, y=3..5
x=503, y=3..5
y=5, x=497..503
x=490, y=8..14
x=510, y=8..14
y=14, x=490..510
x=502, y=10..12
x=505, y=10..12
y=12, x=502..505'''
test_map = parse_map(test_text)
test_filled_map = fill_water(test_map)
print(render_map(test_filled_map))

                              
     4444444444445555555555555
     8899999999990000000000111
     8901234567890123456789012
   0 ............+............
   1 ............|............
   2 ........|||||||||........
   3 ........|#~~~~~#|........
   4 ........|#~~~~~#|........
   5 ........|#######|........
   6 ........|.......|........
   7 .|||||||||||||||||||||||.
   8 .|#~~~~~~~~~~~~~~~~~~~#|.
   9 .|#~~~~~~~~~~~~~~~~~~~#|.
  10 .|#~~~~~~~~~~~#~~#~~~~#|.
  11 .|#~~~~~~~~~~~#~~#~~~~#|.
  12 .|#~~~~~~~~~~~####~~~~#|.
  13 .|#~~~~~~~~~~~~~~~~~~~#|.
  14 .|#####################|.



In [344]:
# Another broken scenario.
test_text = '''x=498, y=10..13
x=502, y=10..13
y=13, x=498..502'''
test_map = parse_map(test_text)
test_filled_map = fill_water(test_map, max_rounds=4)
print(render_map(test_filled_map))
# should be 9 + 8 = 17 (should not count first down stream or water on top layer)
print(count_water(test_filled_map, 10, 13))

              
     444455555
     999900000
     678901234
   0 ....+....
   1 ....|....
   2 ....|....
   3 ....|....
   4 ....|....
   5 ....|....
   6 ....|....
   7 ....|....
   8 ....|....
   9 .|||||||.
  10 .|#~~~#|.
  11 .|#~~~#|.
  12 .|#~~~#|.
  13 .|#####|.

17


In [321]:
with open('input.txt') as input_:
    map_ = parse_map(input_.read().strip())

In [322]:
# For debugging:
with open('output.txt', 'w') as output_:
    output_.write(render_map(map_))

In [323]:
filled_map = fill_water(map_)

In [324]:
# For debugging:
with open('output-filled.txt', 'w') as output_:
    output_.write(render_map(filled_map))

In [327]:
min_y = min(coord[1] for coord, cell in map_.items() if cell != '+')
max_y = max(coord[1] for coord, cell in map_.items() if cell != '+')
print(min_y, max_y)

6 1910


In [328]:
count_water(filled_map, min_y, max_y)

29063

# Part 2